[Rendered Version](http://nbviewer.jupyter.org/github/andersonfrailey/Notebook-Uploads/blob/master/Training%20Doc.ipynb)

In [11]:
from taxcalc import *

This example uses the packaged CPS file. If you were to use the PUF, you would create the records class using
`Records()` or `Records('path to puf')` if the PUF is not in your current working directory.

In [12]:
# Initiate baseline calculator
recs = Records.cps_constructor()
calc = Calculator(records=recs, policy=Policy())
calc.advance_to_year(2018)
calc.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [13]:
# Initiate calculator to apply reforms to
recs_x = Records.cps_constructor()
calc_x = Calculator(records=recs_x, policy=Policy())

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


You can specify your reforms in a JSON file and, using the `read_json_param_objects` method, create a dictionary containing both policy reforms and behavioral assumptions

The policy reform and any behavioral assumptions need to be in separate JSON files.

Here is [more information](https://github.com/open-source-economics/Tax-Calculator/blob/master/taxcalc/reforms/REFORMS.md) on creating reform files.

In [14]:
reforms = calc_x.read_json_param_objects('SampleReform.json', 'JCT_Behavior.json')
reforms

{'behavior': {2018: {u'_BE_inc': [-0.25]}},
 'consumption': {2018: {u'_MPC_e19800': [0.2]}},
 'growdiff_baseline': {},
 'growdiff_response': {},
 'policy': {2018: {u'_ID_RealEstate_hc': [0.25],
   u'_ID_StateLocalTax_hc': [0.25],
   u'_II_rt1': [0.12],
   u'_II_rt2': [0.12],
   u'_II_rt7': [0.35],
   u'_STD': [[12000, 24000, 12000, 18000, 24000]]},
  2019: {u'_ID_RealEstate_hc': [0.5], u'_ID_StateLocalTax_hc': [0.5]},
  2020: {u'_ID_RealEstate_hc': [0.75], u'_ID_StateLocalTax_hc': [0.75]},
  2021: {u'_ID_RealEstate_hc': [1.0], u'_ID_StateLocalTax_hc': [1.0]}}}

Alternatively, you can specify a policy reform as a dictionary. The only drawback with doing this is you will not be able to use the `reform_documentation()` method to print out the reform description.

In [15]:
reform_dict = {
    2018: {
        '__ID_RealEstate_hc': [0.25],
        '_ID_StateLocalTax_hc': [0.25],
        '_II_rt1': [0.12],
        '_II_rt2': [0.12],
        '_II_rt7': [0.35],
        '_STD': [[12000, 24000, 12000, 18000, 24000]]},
    2019: {
        '_ID_RealEstate_hc': [0.5],
        '_ID_StateLocalTax_hc': [0.5]},
    2020: {
        '_ID_RealEstate_hc': [0.75],
        '_ID_StateLocalTax_hc': [0.75]},
    2021: {
        '_ID_RealEstate_hc': [1.],
        '_ID_StateLocalTax_hc': [1.]}
}

In [16]:
calc_x.policy.implement_reform(reforms['policy'])
calc_x.consumption.update_consumption(reforms['consumption'])
calc_x.advance_to_year(2018)

### Behavioral response

A dictionary is also used to implement behavioral reforms. The only difference is you must then pass the baseline and reform calculators into the response method of the behavioral class. This method calculates the change in tax liabilities and then, using the specified elasticities, returns a new calculator object that accounts for any behavioral change.

In [17]:
calc_x.behavior.update_behavior(reforms['behavior'])
calc_response = Behavior.response(calc, calc_x)

### Viewing the results

In [18]:
from taxcalc.utils import *

#### Analyzing Individual Variables

Individual variables are attributes of the records class and can therefore be accessed using a simple dot notation

In [19]:
baseline = (calc.records.combined * calc.records.s006).sum()  # combined is combined tax liability while s006 is weight
reformed = (calc_response.records.combined * calc_response.records.s006).sum()
diff = reformed - baseline
print ('Combined Liability - Baseline: {:0.2f}'.format(baseline))
print ('Combined Liability - Reform: {:>18.2f}'.format(reformed))
print ('-' * 47)
print ('Difference: {:35.2f}'.format(diff))

Combined Liability - Baseline: 2521070511449.64
Combined Liability - Reform:   2408891906085.45
-----------------------------------------------
Difference:                    -112178605364.19


#### Diagnostic Table

Diagnostic tables are the most straight forward methods of evaluation. They simply show aggregate values for a given calculator

In [20]:
create_diagnostic_table(calc)

,2018
Returns (#m),169.9
AGI ($b),"10,503.8"
Itemizers (#m),68.0
Itemized Deduction ($b),"1,500.9"
Standard Deduction Filers (#m),83.9
Standard Deduction ($b),771.4
Personal Exemption ($b),"1,222.6"
Taxable Income ($b),"7,421.9"
Regular Tax ($b),"1,369.7"
AMT Income ($b),"9,664.5"


In [21]:
create_diagnostic_table(calc_response)

,2018
Returns (#m),169.9
AGI ($b),"10,477.2"
Itemizers (#m),29.0
Itemized Deduction ($b),808.7
Standard Deduction Filers (#m),122.9
Standard Deduction ($b),"2,117.0"
Personal Exemption ($b),"1,222.6"
Taxable Income ($b),"7,041.9"
Regular Tax ($b),"1,260.7"
AMT Income ($b),"9,980.7"


#### Distribution Table

The distribution table shows the same information as the diagnostic table, but broken down by income bin or decile. You can view the results as either the weighted average or the weighted sum in each bin

In [22]:
create_distribution_table(calc.records, groupby='weighted_deciles',
                          income_measure='expanded_income', result_type='weighted_avg')

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"16,989,417",-662,1,"6,587",0,5,"5,262",24,1,-664,...,0,1,0,0,98,-97,176,79,-233,-312
1,"16,988,286","5,118",1,"7,814",0,207,"5,904",190,13,"5,018",...,0,13,3,0,755,-745,663,-82,"9,847","9,929"
2,"16,990,004","9,116",1,"7,860",0,"1,045","6,319","1,636",153,"8,514",...,1,154,29,0,"1,020",-895,"1,209",314,"17,245","16,931"
3,"16,989,656","17,018",1,"7,632",0,"2,335","7,151","5,057",532,"15,624",...,2,534,90,0,"1,193",-750,"2,327","1,577","25,262","23,685"
4,"16,989,955","26,269",1,"7,157",0,"3,910","7,667","10,826","1,244","23,898",...,4,"1,247",168,0,921,158,"3,609","3,768","34,401","30,633"
5,"16,989,241","37,913",1,"6,375",0,"5,941","8,090","19,220","2,298","34,325",...,5,"2,303",279,0,426,"1,598","5,182","6,780","45,965","39,185"
6,"16,989,686","54,723",0,"4,842",1,"9,622","8,499","32,350","4,206","48,877",...,3,"4,209",394,0,82,"3,733","7,416","11,149","61,857","50,708"
7,"16,989,485","78,655",0,"3,383",1,"13,995","9,343","52,170","7,544","70,468",...,4,"7,549",496,0,13,"7,040","10,754","17,794","85,684","67,890"
8,"16,989,613","114,892",0,"2,039",1,"19,742","10,530","82,679","12,939","103,702",...,6,"12,946",449,0,3,"12,494","15,860","28,354","124,252","95,898"
9,"16,989,497","275,212",0,990,1,"31,544","10,053","232,701","51,691","259,091",...,"1,036","52,726",29,549,0,"53,246","25,411","78,657","290,513","211,856"


In [23]:
create_distribution_table(calc_response.records, groupby='weighted_deciles',
                          income_measure='expanded_income', result_type='weighted_sum')

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"16,989,417","-11,240,375,934","9,514,456","195,449,294,145","11,007","78,047,070","89,401,300,451","408,960,592","14,266,689","-11,276,018,050",...,0,"14,266,689","7,372,907",0,"1,657,208,090","-1,650,314,307","2,997,688,406","1,347,374,099","-3,954,094,409","-5,301,468,508"
1,"16,989,094","86,942,793,611","11,743,649","240,153,684,756","14,673","135,713,300","100,269,230,549","1,201,206,381","38,077,774","86,885,674,235",...,"4,712,568","42,790,342","22,410,102",0,"12,832,180,654","-12,811,800,414","11,264,576,633","-1,547,223,780","167,252,301,579","168,799,525,359"
2,"16,989,774","154,899,070,041","13,250,924","253,626,171,467","168,243","3,168,282,200","106,944,558,140","5,579,806,223","566,732,228","152,868,950,679",...,"15,360,182","582,092,409","110,053,030",0,"17,521,776,767","-17,049,737,388","20,553,723,149","3,503,985,761","292,470,862,332","288,966,876,571"
3,"16,988,865","288,334,364,834","15,143,372","269,448,059,736","592,410","10,883,349,440","121,575,274,041","47,175,058,421","5,306,510,960","281,295,598,040",...,"38,392,688","5,344,903,648","838,128,862",0,"21,027,651,163","-16,520,876,377","39,451,196,246","22,930,319,869","428,221,707,216","405,291,387,347"
4,"16,990,013","444,704,994,441","15,619,435","278,597,274,674","967,115","17,825,971,960","130,135,786,463","123,628,950,403","14,069,474,314","433,109,364,110",...,"79,220,613","14,148,694,926","1,821,035,652",0,"16,688,106,892","-4,360,447,617","61,132,682,388","56,772,234,770","582,666,670,640","525,894,435,870"
5,"16,989,624","643,435,399,906","15,328,972","280,060,128,859","1,564,257","32,581,640,674","137,969,239,408","247,348,392,312","28,248,830,646","622,300,542,104",...,"103,822,902","28,352,653,548","3,302,265,340",0,"8,921,296,924","16,129,091,284","87,988,164,705","104,117,255,989","778,155,990,660","674,038,734,671"
6,"16,989,348","923,035,628,950","14,321,210","274,000,321,776","2,649,184","58,433,658,339","144,180,090,735","462,442,718,292","55,181,760,294","885,798,622,478",...,"103,709,690","55,285,469,984","6,036,857,634",0,"2,009,988,084","47,238,624,266","124,991,931,166","172,230,555,432","1,046,775,342,352","874,544,786,920"
7,"16,989,375","1,331,578,390,474","12,884,653","267,245,637,532","4,101,801","97,598,641,939","158,962,857,293","811,437,972,310","109,124,128,582","1,270,123,094,836",...,"175,072,373","109,299,200,955","8,417,666,651",0,"251,138,665","100,630,395,640","182,074,955,587","282,705,351,227","1,450,900,603,142","1,168,195,251,915"
8,"16,989,663","1,945,710,099,591","10,318,033","229,070,326,429","6,668,977","178,032,517,125","178,899,911,039","1,360,970,193,679","194,412,120,481","1,832,439,223,410",...,"259,964,433","194,672,084,913","7,696,744,670",0,"44,455,337","186,930,884,907","268,688,455,293","455,619,340,199","2,104,363,692,660","1,648,744,352,461"
9,"16,989,668","4,669,836,106,937","4,758,822","108,185,831,895","12,230,510","409,939,288,796","170,736,582,929","3,981,737,157,467","853,753,127,928","4,427,147,759,309",...,"17,463,462,109","871,216,590,037","492,692,087","9,298,398,306","3,075,346","880,019,220,909","431,193,491,610","1,311,212,712,520","4,929,555,802,184","3,618,343,089,665"


#### Differences Table

The differences table displays the difference between your baseline and refoms. You can also group the results by decile or income bin.

In [24]:
create_difference_table(calc.records, calc_response.records, groupby='weighted_deciles',
                        income_measure='expanded_income', tax_to_diff='combined')

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,416.58",0.00,0.00,"77,865.94",0.46,0.07,"1,193,396.44",-0.00,0.00,-0.00
1,"16,988,285.70","1,403,423.04",8.26,"47,446.98",0.28,-10.58,"-179,704,079.68",0.16,-0.10,0.07
2,"16,990,004.05","5,050,748.79",29.73,"64,533.66",0.38,-121.64,"-2,066,585,817.15",1.84,-0.79,0.59
3,"16,989,656.05","8,009,175.29",47.14,"683,007.70",4.02,-230.20,"-3,910,998,318.74",3.49,-1.06,0.80
4,"16,989,954.66","10,878,167.44",64.03,"983,313.73",5.79,-433.38,"-7,363,181,927.97",6.56,-1.51,1.14
5,"16,989,240.80","13,085,117.32",77.02,"1,090,948.11",6.42,-664.91,"-11,296,355,312.29",10.07,-1.78,1.34
6,"16,989,686.38","14,605,299.49",85.97,"1,339,801.24",7.89,-981.44,"-16,674,384,517.30",14.86,-2.01,1.51
7,"16,989,484.52","15,353,423.55",90.37,"1,400,588.70",8.24,"-1,141.77","-19,398,072,372.39",17.29,-1.73,1.30
8,"16,989,613.44","16,093,942.29",94.73,"797,506.37",4.69,"-1,541.99","-26,197,883,909.73",23.35,-1.63,1.22
9,"16,989,497.48","11,790,903.24",69.40,"1,819,890.78",10.71,"-1,476.95","-25,092,632,505.38",22.37,-0.63,0.47


#### Plotting

You can use built in methods to get MTR and ATR plots. Each one is returned as a simple Bokeh figure that you can then add to as desired

In [25]:
from bokeh.io import show, output_notebook
output_notebook()

Loading BokehJS ...

In [26]:
mtr_plot_data = mtr_graph_data(calc, calc_response)

In [27]:
show(xtr_graph_plot(mtr_plot_data))

In [28]:
atr_plot_data = atr_graph_data(calc, calc_response)

In [29]:
show(xtr_graph_plot(atr_plot_data))

#### Multiyear diagnostic tables

You can also produce diagnostic tables up to 2026

In [30]:
multiyear_diagnostic_table(calc, num_years=9)

,2018,2019,2020,2021,2022,2023,2024,2025,2026
Returns (#m),169.9,172.2,174.5,176.9,179.3,181.7,184.1,186.6,189.1
AGI ($b),"10,503.8","10,987.4","11,474.4","11,999.4","12,561.1","13,146.7","13,759.3","14,401.2","15,070.1"
Itemizers (#m),68.0,69.6,71.2,72.7,74.2,75.9,77.5,79.1,80.8
Itemized Deduction ($b),"1,500.9","1,585.9","1,672.3","1,765.4","1,863.4","1,967.0","2,075.8","2,189.2","2,312.4"
Standard Deduction Filers (#m),83.9,84.3,84.8,85.4,86.0,86.5,87.1,87.7,88.3
Standard Deduction ($b),771.4,791.5,813.9,838.1,863.3,887.8,914.4,940.9,968.1
Personal Exemption ($b),"1,222.6","1,266.2","1,312.4","1,361.2","1,412.5","1,464.9","1,519.7","1,576.1","1,634.7"
Taxable Income ($b),"7,421.9","7,768.6","8,113.2","8,485.3","8,886.3","9,305.2","9,742.7","10,203.3","10,678.8"
Regular Tax ($b),"1,369.7","1,442.9","1,514.0","1,590.9","1,672.5","1,756.7","1,843.9","1,935.8","2,029.1"
AMT Income ($b),"9,664.5","10,099.5","10,536.6","11,008.0","11,513.5","12,039.3","12,589.2","13,165.8","13,763.9"


In [31]:
multiyear_diagnostic_table(calc_response, num_years=9)

,2018,2019,2020,2021,2022,2023,2024,2025,2026
Returns (#m),169.9,172.2,174.5,176.9,179.3,181.7,184.1,186.6,189.1
AGI ($b),"10,477.2","10,959.6","11,445.2","11,968.7","12,528.8","13,112.8","13,723.8","14,364.1","15,031.2"
Itemizers (#m),29.0,25.1,20.9,17.3,18.1,18.9,19.8,20.7,21.7
Itemized Deduction ($b),808.7,678.5,542.5,423.0,457.2,493.1,532.0,573.8,621.1
Standard Deduction Filers (#m),122.9,128.8,135.1,140.8,142.1,143.5,144.8,146.2,147.4
Standard Deduction ($b),"2,117.0","2,280.0","2,459.2","2,638.7","2,726.2","2,815.1","2,907.4","3,001.2","3,096.2"
Personal Exemption ($b),"1,222.6","1,266.2","1,312.4","1,361.2","1,412.5","1,464.9","1,519.7","1,576.1","1,634.7"
Taxable Income ($b),"7,041.9","7,465.1","7,880.7","8,315.8","8,725.7","9,154.7","9,603.3","10,075.1","10,565.3"
Regular Tax ($b),"1,260.7","1,351.3","1,438.9","1,529.8","1,611.1","1,695.3","1,782.7","1,874.7","1,969.3"
AMT Income ($b),"9,980.7","10,491.5","11,019.7","11,594.8","12,124.4","12,676.2","13,252.3","13,855.0","14,479.6"


#### Reporting your changes

To display what reforms you included in a way that is easy for humans to read, you can use the `reform_documentation` method. It will print out all of the policy parameters you've specified with a short description and their default and current values


_Note: this feature is not yet available in the taxcalc package. You must use the source code to access it. It will be available in the next taxcalc release._

In [32]:
print(calc_response.reform_documentation(reforms))

REFORM DOCUMENTATION
Baseline Growth-Difference Assumption Values by Year:
none: using default baseline growth assumptions
Policy Reform Parameter Values by Year:
2018:
 _ID_RealEstate_hc : 0.25
  name: State, local, and foreign real estate taxes deduction haircut.
  desc: This decimal fraction reduces real estate taxes paid eligible to
        deduct in itemized deduction.
  baseline_value: 0.0
 _ID_StateLocalTax_hc : 0.25
  name: State and local income and sales taxes deduction haircut.
  desc: This decimal fraction reduces the state and local income and sales tax
        deduction.
  baseline_value: 0.0
 _II_rt1 : 0.12
  name: Personal income (regular/non-AMT/non-pass-through) tax rate 1
  desc: The lowest tax rate, applied to the portion of taxable income below
        tax bracket 1.
  baseline_value: 0.1
 _II_rt2 : 0.12
  name: Personal income (regular/non-AMT/non-pass-through) tax rate 2
  desc: The second lowest tax rate, applied to the portion of taxable income
        below ta

One thing I didn't cover is how to extract the marginal tax rates. All you need to do is call the `Calculator.mtr()` method. It will return MTR for individual income tax, payroll tax, and individaul income tax + payroll tax

In [33]:
mtr_payroll, mtr_income, mtr_combined = calc.mtr()

In [34]:
mtr_payroll

array([ 0.14212726,  0.14212726,  0.14212726, ...,  0.14212726,
        0.14212726,  0.14212726])

In [35]:
mtr_income

array([ 0.13934046,  0.        ,  0.        , ...,  0.23687877,
        0.23687877,  0.23687877])

In [36]:
mtr_combined

array([ 0.28146772,  0.14212726,  0.14212726, ...,  0.37900604,
        0.37900604,  0.37900604])